In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%config Completer.use_jedi = False

In [ ]:
from siasearch import SiaSearch

In [ ]:
!pip install siasearch

# Setup

We start by importing the authentication function auth and creating a sia object that connects to our public API

In [ ]:
sia = SiaSearch.login("https://demo.sia-search.com", "user@siasearch.io", "password")
sia

# Columns

By interacting with the `SiaSearch` object we can get information about all our columns and the specific properties

In [ ]:
cols = sia.columns()
cols

Here we fetch a numeric column, that has multiple different properties e.g. unit

In [ ]:
cols.forward_velocity

In [ ]:
cols.forward_velocity.unit

# Query

By interacting with the `SiaSearch` object we can get information about all our columns and the specific properties

It returns a `Query` object

In [ ]:
query = sia.query("dataset_name = 'mxa' AND curved_trajectory = 'LEFT_BEND' AND forward_velocity > 10")
query

The actual query results are fetched by calling `Query.all()`

In [ ]:
query.all()

# Sequences

Queries return `Sequence` objects

In [ ]:
sequences = query.all()
sequences

`Sequences` can be combined and sliced

In [ ]:
sequences[0:2] + sequences[5:]

Sequences can be viewed in the frontend if they have a corresponding query by calling `view`

In [ ]:
sequences.view()

A single element in `Sequences` is a `Sequence`. It is defined by `start_timestamp`, `end_timestamp` and a `drive_id`

In [ ]:
seq = sequences[5]
seq

A sequence can also be viewed

In [ ]:
seq.view()

The underlying data can also be fetched into a `ColumnValues` object

In [ ]:
col_values = seq.column_values(["forward_velocity", "curved_trajectory"])
col_values.to_pandas()

# Frames

A `Frames` class is used to work with individual frames within each sequence and is derived from `Sequences` and `Sequence` objects

In [ ]:
frames = sequences.frames()
frames

`Frames` can be sliced and combined as well

In [ ]:
frames_subset = frames[10:20] + frames[50:70]
len(frames_subset)

Frames can be subsampled in time and number of samples per drive

In [ ]:
frames.subsample(min_time_between_samples_per_drive=1).to_pandas()

In [ ]:
frames_subsampled = frames.subsample(num_samples_per_drive=2)
frames_subsampled.to_pandas()

`Frames` consist of individual `Frame` objects

In [ ]:
frame = frames[10]
frame.plot()

In [ ]:
bboxes = frame.bboxes(model_name="yolo-928-0-5")
bboxes

`plot` also takes bounding boxes as arguments and overlays them on top of the image

In [ ]:
frame.plot(bboxes)

# Evaluation

In [ ]:
sia.model_evaluation.available_models(frames)

In [ ]:
model_eval_obj_det = sia.model_evaluation.object_detection(frames, "gt", "yolo-608-0-3")
model_eval_obj_det

In [ ]:
model_eval_obj_det.evaluate(iou_threshold=0.5)